In [19]:
import torch
import numpy as np
import pandas as pd
from torch import nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [20]:
if torch.cuda.is_available():
    train_on_gpu = True
    print("Gpu available")
else:
    train_on_gpu = False
    print("Gpu not available")

Gpu available


In [21]:
class NN(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(NN, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, 64)
        self.fc2 = torch.nn.Linear(64, 32)
        self.fc3 = torch.nn.Linear(32,16)
        self.fc4 = torch.nn.Linear(16, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.softmax(self.fc4(x))
        return x
    
input_dim = 51
output_dim  = 3
batch_size = 64
model = NN(input_dim, output_dim)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model

NN(
  (fc1): Linear(in_features=51, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=3, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)

In [22]:
class CustomLoader(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.dataframe = pd.read_csv(csv_file)
        self.np_input_X = self.dataframe[['Score_1', 'X_pos_1', ' Y_pos_1', ' Score_2', 'X_pos_2', ' Y_pos_2',
                                   ' Score_3', 'X_pos_3', ' Y_pos_3', ' Score_4', 'X_pos_4', ' Y_pos_4',
                                   ' Score_5', 'X_pos_5', ' Y_pos_5', ' Score_6', 'X_pos_6', ' Y_pos_6',
                                   ' Score_7', 'X_pos_7', ' Y_pos_7', ' Score_8', 'X_pos_8', ' Y_pos_8',
                                   ' Score_9', 'X_pos_9', ' Y_pos_9', ' Score_10', 'X_pos_10', ' Y_pos_10',
                                   ' Score_11', 'X_pos_11', ' Y_pos_11', ' Score_12', 'X_pos_12',
                                   ' Y_pos_12', ' Score_13', 'X_pos_13', ' Y_pos_13', ' Score_14',
                                   'X_pos_14', ' Y_pos_14', ' Score_15', 'X_pos_15', ' Y_pos_15',
                                   ' Score_16', 'X_pos_16', ' Y_pos_16', ' Score_17', 'X_pos_17',
                                   ' Y_pos_17']].to_numpy(dtype=np.float32)
        
        self.np_input_y = self.dataframe['mapped_output'].to_numpy()
        

        
#         self.input_X = (self.np_input_X).ToTensor()
#         self.input_y = torch.from_numpy(self.np_input_y)
        
    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        self.input_X = torch.from_numpy(self.np_input_X)
        self.input_y = torch.from_numpy(self.np_input_y)
        
        sample = {'input': self.input_X[idx], 'output': self.input_y[idx]}
        return sample
    
trainset = CustomLoader(csv_file='Final/train.csv')
testset = CustomLoader(csv_file='Final/test.csv')

# TO Making sure data is loaded
# for i in range(len(trainset)):
#     sample = trainset[i]
#     print(sample['input'], "\n", sample['output'])
#     break
# type(sample['input'])

In [23]:
valid_size = 0.2
num_train = len(trainset)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [24]:
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size)

In [25]:
model.load_state_dict(torch.load('output/posenet.pt'))
dummy_input = Variable(torch.randn(1, 51))
torch.onnx.export(model, dummy_input, "output/posenet.onnx")

In [26]:
import onnx
from onnx_tf.backend import prepare

# Load the ONNX file
model = onnx.load('output/posenet.onnx')

# Import the ONNX model to Tensorflow
tf_rep = prepare(model)

In [27]:
# # Input nodes to the model
# print('inputs:', tf_rep.inputs)

# # Output nodes from the model
# print('outputs:', tf_rep.outputs)

# # All nodes in the model
# print('tensor_dict:')
# print(tf_rep.tensor_dict)

In [29]:
for content in testloader:
    # move tensors to GPU if CUDA is available
    data = content['input']
    target = content['output']
    break

In [30]:
output = tf_rep.run(data.cpu())
torch.max(torch.from_numpy(output[0]), 1)[1]

tensor([0, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 0, 0, 0, 2, 2, 0, 2, 2, 1, 0, 0, 1, 1,
        0, 2, 1, 1, 1, 0, 0, 1, 1, 2, 2, 1, 2, 1, 2, 0, 0, 1, 2, 2, 1, 1, 1, 1,
        0, 2, 0, 0, 0, 1, 1, 2, 2, 2, 2, 0, 2, 0, 2, 2])

In [41]:
output = tf_rep.run(data[2].view(1,51).cpu())
torch.max(torch.from_numpy(output[0]), 1)[1].item()

1